# پیش بینی امتیاز داده شده به یک محصول براساس کامنت های داده شده

In [1]:
from sklearn.preprocessing import normalize, LabelEncoder
from sklearn.cluster import KMeans
from numpy import unique
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc
from numpy import where
from sklearn.neighbors import KernelDensity
from scipy.signal import argrelextrema
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from google_trans_new import google_translator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [2]:
keifiat = pd.read_excel('E:/university/data mining/dataset_2/keifiat.xlsx')
keifiat

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,549845,شید آباژور آرام مدل SH45/01,TC,4979401,0,0,verified,recommended,بسیار تمیز و زیبا,بسیار تمیز و زیبا...من که خیلی راضیم,"[""کاملا مطابق با عکس\r"",""ساخت بسیار تمیز""]","[""ندارد""]"
99996,784160,کفش مخصوص دویدن مردانه و زنانه آلبرتینی مدل دا...,FA,1066034,7,32,verified,\N,توصیه به خرید میکنم,من این کفش را برای پیاده روی اربعین خریدم از ی...,"[""سبک زیبا ارزان ""]","[""محل عبور بند کفش ها ایراد دارد""]"
99997,820985,واکس کفش ساتل مدل 20 حجم 60 میلی لیتر,FA,568208,0,0,verified,recommended,خوبه و غلیظه,واکس خوبیه..,"[""روغنیه""]","[""ندیدم""]"
99998,678073,کش رنگی مدادچوبی مدل 010 بسته 100,TC,981594,2,1,verified,no_idea,سلام,هیچ کدوم از این رنگهای که توی عکسه داخل بسته ن...,NaN,NaN


In [3]:
keifiat=keifiat.filter(['product_id','comment'])

In [4]:
keifiat

,product_id,comment
0,3692,واقعا عالیه. من که ازش خیلی راضیم
1,90213,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
2,59473,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...
3,120499,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...
4,67200,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...
...,...,...
99995,549845,بسیار تمیز و زیبا...من که خیلی راضیم
99996,784160,من این کفش را برای پیاده روی اربعین خریدم از ی...
99997,820985,واکس خوبیه..
99998,678073,هیچ کدوم از این رنگهای که توی عکسه داخل بسته ن...


# my laptop isnt very good and couldnt translate and process huge numbers of comments so i had to reduce them

In [5]:
index_names = keifiat[keifiat['product_id'] <890000].index 
# drop these given row 
# indexes from dataFrame 
keifiat.drop(index_names,inplace = True) 

In [6]:
keifiat

,product_id,comment
89445,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...
89485,891034,تحویل بموقع و در دسترس بودن خدمات
89753,891045,گرونه وبه بازار ایران دیر میرسه
89969,891312,این کاپشن از جنس بیرونی فوق العاده مناسب در بر...
90202,890725,عالیه واقعا یکی از بهترین صندلیا میتونم بگم هست
...,...,...
99934,907654,کیفیتش خوبه قیمتش هم مناسبه
99935,907647,کیفیتش خوبه قیمتش هم مناسبه
99936,907634,کیفیتش خوبه قیمتش هم مناسبه
99952,908760,من فیکورس مدلهای اونتوس و وود رو دارم، انصافا ...


# first we translate and then do NLP with python-Text Clustering

In [7]:
trans = google_translator()  

In [8]:
translate_text=[]
for comment in keifiat['comment']:
    txt=trans.translate(comment, lang_src='fa', lang_tgt='en')
    print(txt)
    translate_text.append(txt)  

Hi friends, this phone is great in terms of hardware, very, very strong, especially the 4200 amp battery, it is also very handy and beautiful, it has a very good camera, it is also very economical. 
Timely delivery and availability of services 
Expensive and late to the Iranian market 
This jacket is made of outer material that is extremely suitable for rain and this quality seems very good at this price. 
Alia is really one of the best chairs I can say 
I received it yesterday wonderful 
The best phone in the world right now to play 
Hello friends, Huawei's new flagship is a powerful high-performance phone that few can complain about. There is only one problem that with this currency situation in Iran, it is not possible for most of us to buy it. I suggest those who can afford it to buy it. .  Of course, I hope that the currency problem will be solved by the time this product is released 
I bought this brand twice. I got from customs.  I really say just playing with human souls ... 
I

In [9]:
translate_text

['Hi friends, this phone is great in terms of hardware, very, very strong, especially the 4200 amp battery, it is also very handy and beautiful, it has a very good camera, it is also very economical. ',
 'Timely delivery and availability of services ',
 'Expensive and late to the Iranian market ',
 'This jacket is made of outer material that is extremely suitable for rain and this quality seems very good at this price. ',
 'Alia is really one of the best chairs I can say ',
 'I received it yesterday wonderful ',
 'The best phone in the world right now to play ',
 "Hello friends, Huawei's new flagship is a powerful high-performance phone that few can complain about. There is only one problem that with this currency situation in Iran, it is not possible for most of us to buy it. I suggest those who can afford it to buy it. .  Of course, I hope that the currency problem will be solved by the time this product is released ",
 'I bought this brand twice. I got from customs.  I really say ju

In [10]:
keifiat['commenten']=translate_text

In [11]:
keifiat['commenten']

89445    Hi friends, this phone is great in terms of ha...
89485        Timely delivery and availability of services 
89753            Expensive and late to the Iranian market 
89969    This jacket is made of outer material that is ...
90202     Alia is really one of the best chairs I can say 
                               ...                        
99934     Its quality is good and its price is reasonable 
99935     Its quality is good and its price is reasonable 
99936     Its quality is good and its price is reasonable 
99952    I have the Ficors of Ontus and Wood models, in...
99992    We used it, it was good, I was looking for it ...
Name: commenten, Length: 121, dtype: object

In [12]:
keifiat2=keifiat
keifiat2

,product_id,comment,commenten
89445,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...,"Hi friends, this phone is great in terms of ha..."
89485,891034,تحویل بموقع و در دسترس بودن خدمات,Timely delivery and availability of services
89753,891045,گرونه وبه بازار ایران دیر میرسه,Expensive and late to the Iranian market
89969,891312,این کاپشن از جنس بیرونی فوق العاده مناسب در بر...,This jacket is made of outer material that is ...
90202,890725,عالیه واقعا یکی از بهترین صندلیا میتونم بگم هست,Alia is really one of the best chairs I can say
...,...,...,...
99934,907654,کیفیتش خوبه قیمتش هم مناسبه,Its quality is good and its price is reasonable
99935,907647,کیفیتش خوبه قیمتش هم مناسبه,Its quality is good and its price is reasonable
99936,907634,کیفیتش خوبه قیمتش هم مناسبه,Its quality is good and its price is reasonable
99952,908760,من فیکورس مدلهای اونتوس و وود رو دارم، انصافا ...,"I have the Ficors of Ontus and Wood models, in..."


# Text preprocessing Steps
# ●Convert text to lowercase
# ●Remove numbers
# ●Remove punctuation
# ●Remove whitespaces
# ●Tokenization
# ●Remove stop words
# ●Stemming
# ●Lemmatization

In [13]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

def lemmatize(s):
    lemmatizer = WordNetLemmatizer()
    s = ' '.join([lemmatizer.lemmatize(word) for word in s])
    return s

def Convert_text_to_lowercase(s):
    s = s.lower()
    return s

def Remove_numbers(s):
    s = s.replace('\d+', '')
    return s

def Remove_whitespaces(s):
    s = s.replace('[^\w\s]','')
    s = s.strip()
    return s

def Tokenization(s):
    s = nltk.word_tokenize(s)
    return s

def Remove_stop_words(s):
    stop = set(stopwords.words('english')) - set(['against','any','not','very','should','will','no','nor','below','above'])
    s= [item for item in s if item not in stop]
    return s

def Stemming(s):
    stemmer = SnowballStemmer("english")
    s = [stemmer.stem(y) for y in s] # Stem every word.
    return s

In [14]:
tex=[]
for index in keifiat2.index:
    tex_row= keifiat2['commenten'][index]
    tex_row=Convert_text_to_lowercase(tex_row)
    tex_row=Remove_numbers(tex_row)
    tex_row=remove_punctuation(tex_row)
    tex_row=Remove_whitespaces(tex_row)
    tex_row=Tokenization(tex_row)
    tex_row=Remove_stop_words(tex_row)
    tex_row=Stemming(tex_row)
    tex_row=lemmatize(tex_row)
    tex.append(tex_row)

keifiat2['commenten']=tex
scaled_features=keifiat2['commenten'].to_numpy()
scaled_features

array(['hi friend phone great term hardwar veri veri strong especi 4200 amp batteri also veri handi beauti veri good camera also veri econom',
       'time deliveri avail servic', 'expens late iranian market',
       'jacket made outer materi extrem suitabl rain qualiti seem veri good price',
       'alia realli one best chair say', 'receiv yesterday wonder',
       'best phone world right play',
       'hello friend huawei new flagship power highperform phone complain one problem currenc situat iran not possibl u buy suggest afford buy cours hope currenc problem will solv time product releas',
       'bought brand twice got custom realli say play human soul',
       'veri good appear great price close exampl may even weaker',
       'bought refriger compani newest product snow compani that lot work everyth almost equal side refriger samsung lg beauti design cost lot expens compar korean brand much cheaper',
       'book not suitabl peopl not know distinguish healthi narrat fake one co

# i tried some values for numbers of clusters and realized 10 was okay

In [17]:
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,3))
vectorizer.fit(keifiat['commenten'].values)
feautures=vectorizer.transform(keifiat['commenten'].values)
clust = KMeans(max_iter=100,n_clusters=10,n_init=10,init='k-means++')
clust.fit(feautures)

KMeans(max_iter=100, n_clusters=10)

In [18]:
yhat=clust.predict(feautures)
keifiat['cluster labels']=clust.labels_
keifiat

,product_id,comment,commenten,cluster labels
89445,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...,hi friend phone great term hardwar veri veri s...,1
89485,891034,تحویل بموقع و در دسترس بودن خدمات,time deliveri avail servic,5
89753,891045,گرونه وبه بازار ایران دیر میرسه,expens late iranian market,7
89969,891312,این کاپشن از جنس بیرونی فوق العاده مناسب در بر...,jacket made outer materi extrem suitabl rain q...,1
90202,890725,عالیه واقعا یکی از بهترین صندلیا میتونم بگم هست,alia realli one best chair say,7
...,...,...,...,...
99934,907654,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8
99935,907647,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8
99936,907634,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8
99952,908760,من فیکورس مدلهای اونتوس و وود رو دارم، انصافا ...,ficor ontus wood model fair bush allencompass ...,1


In [19]:
dat=keifiat[(keifiat['cluster labels']==0)]
dat

,product_id,comment,commenten,cluster labels
91292,894711,دیروز به دستم رسید فوق العادست,receiv yesterday wonder,0
93807,890656,جدیدترینCODتابه حال,latest cod far,0
95882,900305,امروز بدستم رسید. جنسش از فلز برنز هست و فکر ن...,got today made bronz metal not think would heavi,0
96998,892295,دارای الیاف نخی دون دون شونده,contain declin cotton fiber,0
97433,891432,فوق العاده است,fantast,0
97496,896697,درست همین چیزی که تو عکس میبینیدو میارن هم کول...,exact see pictur backpack handbag detach,0
98567,896290,همین دیگه موفق باشید,good luck,0


In [20]:
dat=keifiat[(keifiat['cluster labels']==1)]
dat

,product_id,comment,commenten,cluster labels
89445,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...,hi friend phone great term hardwar veri veri s...,1
89969,891312,این کاپشن از جنس بیرونی فوق العاده مناسب در بر...,jacket made outer materi extrem suitabl rain q...,1
92062,894350,ظاهر خیلی خوبی داره که نسبت به این قیمت فوق ال...,veri good appear great price close exampl may ...,1
93268,890656,خدایش خیلی باحاله به نظر من خیلی باحال هست اما...,god veri cool think veri cool also veri expens,1
94608,891635,قسمت بازو آزاد این چرخ خیلی کارو راحت کرده دی...,free arm part wheel made veri easi work,1
94929,900065,از خریدم راضی هستم \r\nفشار خوبی داره طول شلنگ...,satisfi purchas good pressur length hose suita...,1
94946,900970,من تعداد زیادی از بدلیجات رو خریدم و تقریبا هر...,bought lot rhineston use almost everi day good...,1
94958,900873,روی لباسهای با رنگ بنفش، زرد، سبز و حتی مشکی خ...,look great cloth purpl yellow green even black...,1
95101,896651,خمیر خوب و مناسبیه اگه اورکلاکر نیستین الکی پو...,dough good suitabl,1
95877,901032,علیرغم زیبایی و طرح جدید، قیمت مناسبی داره.,despit beauti new design reason price,1


In [21]:
dat=keifiat[(keifiat['cluster labels']==2)]
dat

,product_id,comment,commenten,cluster labels
92260,896520,این کتاب برای افرادی که علم تشخیص روایات سالم ...,book not suitabl peopl not know distinguish he...,2
92521,890864,محصولات سمیه رو دوستش دارم,like somayeh product,2
95954,900652,من این کالا را وقتی دیگر چیزی با قیمت مناسب بر...,bought product noth reason price offer market ...,2
97585,906128,من الان این محصول اینجا دیدم\r\nلازم دونستم به...,saw product someon use least year say excel im...,2
99858,908646,با توجه به اینکه ارتفاع طبقات 30 سانتی متریه، ...,due fact height floor 30 cm suitabl big book l...,2


In [22]:
dat=keifiat[(keifiat['cluster labels']==3)]
dat

,product_id,comment,commenten,cluster labels
92873,895394,برای کسایی که به این فیلد علاقه دارن مفیده. تر...,use interest field translat fluent understand,3
93511,890656,عالی و بی نظیر,excel uniqu,3
94696,896621,من مدل s970 رو الان دارم بسیار عالی و باکیفیت ...,s970 model excel high qualiti complet professi...,3
94922,900065,سلام دوستان من این کارواشو تهیه کردم نسبت به ه...,hi friend made car wash better featur advantag...,3
94991,900065,دسته تلسکوپی برای حمل و نقل آسان دارد. طراحی ز...,telescop handl easi transport beauti design po...,3
96078,890577,سایز من لارج هست ولی ایکس لارج خریدم ولی انگار...,size larg bought xlarg look like medium mean s...,3
96187,902565,عالی,excel,3
97645,897775,سلام.این مکمل رو روی ماشین دنا استفاده کردم شت...,hi use supplement dena car give great acceler ...,3
97782,892480,واقعا بدرد نخور...,dont realli use,3
98217,899757,دستمال ساخت کره هست و کیفیتش عالیه بدون آب هم ...,napkin made butter qualiti excel without water,3


In [23]:
dat=keifiat[(keifiat['cluster labels']==4)]
dat

,product_id,comment,commenten,cluster labels
95914,900929,با این که از استیل هست خیلی ظریف طراحی شده .طر...,although made steel veri delic design design e...,4
96233,903089,تلفیقی از طراحی مدرن بر روی صنایع دستی ایرانی,combin modern design iranian handicraft,4
96773,899888,بسیار زیبا و شیک ،خیلی خاصه,veri beauti stylish veri special,4
97282,903482,نیم بوت خیلی راحتیه و بسیار زیبا تو پا خودشو ...,ankl boot veri comfort veri beauti,4
97306,903494,بسیار زیبا و طراحی خیلی خوبی هم داره,veri beauti veri good design,4
98626,890738,واقعا طراحی زیبایی داره من شیفته طرح های روی چ...,realli beauti design fell love design leather,4
98763,893165,خیلی نرم و لطیفه\r\nو بسیار زیباست\r\nسبک و گر...,veri soft gentl veri beauti also light warm wa...,4
98767,893116,خیلی خیلی قشنگه. خیلییییی نرمه. واقعا عالی کام...,veri veri beauti veri soft realli great comple...,4
99733,891504,خیلی قشنگ و ظریفه فقط کاش گوشواره هم داشت که م...,veri beauti eleg wish earring could set,4


In [24]:
dat=keifiat[(keifiat['cluster labels']==5)]
dat

,product_id,comment,commenten,cluster labels
89485,891034,تحویل بموقع و در دسترس بودن خدمات,time deliveri avail servic,5
91358,894437,بهترین گوشی حال حاضر جهان برای بازی,best phone world right play,5
91996,893884,سلام دوستان، پرچم دار جدید هواوی یک گوشی پرقدر...,hello friend huawei new flagship power highper...,5
93348,893884,اگر کسی توان مالی برای خرید این گوشی را دارد بخرد,anyon afford buy phone,5
94433,893884,سلام دوستان این گوشی فوق العاده است و بعضیا می...,hello friend phone great peopl say price good,5
98106,893884,حییییییف پولش نیست\r\nهرکی پولشو داره بخره,no money whoever money buy,5
98577,894117,احتمال شکستن شیشه ال سی دی,possibl break lcd glass,5
98800,900428,با یکبار افتادن گوشی قاب خورد میشه,phone drop frame crash,5
99502,908917,اگه اصل باشه ارزش خرید را داره,origin worth buy,5


In [25]:
dat=keifiat[(keifiat['cluster labels']==6)]
dat

,product_id,comment,commenten,cluster labels
92055,895299,من از این مارک 2 بار خرید. کردم از گمرک.\r\nوا...,bought brand twice got custom realli say play ...,6
92198,892534,من این یخچال رو از شرکت خریداری کردم.....جدیدت...,bought refriger compani newest product snow co...,6
93064,894556,برند بسیار عالی هستش من چندین مدل از چند سال ق...,excel brand sever model year ago satisfi quali...,6
96055,903442,من از داشتن این پاور بانک راضی هستم و باریک تر...,satisfi power bank slimmest power bank world 1...,6
96238,903151,نقطه کوبی بسیار زیبا با رنگ عالی,stun dot great color,6
96241,902418,سلام \r\nکفش سی سی یک برند 70 ساله و شناخته شد...,hi cc shoe 70yearold brand known field leather...,6
97452,903832,ساعت سیکو ژاپن را با اطمینان کامل باید خرید ! ...,should buy seiko japan watch complet confid se...,6
97725,897467,پیشنهاد میکنم قبل از خرید لوگوی پشت ساعت رو با...,buy suggest compar logo back watch photo googl...,6
99242,892894,من این طناب رو ده ماهه دارم استفاده میکنم کیفی...,use rope ten month definit suggest buy excel q...,6
99570,900624,کیفیت این برند خیلی خوبه ونسبت به مارک های دیگ...,qualiti brand veri good beauti cashmer better ...,6


In [26]:
dat=keifiat[(keifiat['cluster labels']==7)]
dat

,product_id,comment,commenten,cluster labels
89753,891045,گرونه وبه بازار ایران دیر میرسه,expens late iranian market,7
90202,890725,عالیه واقعا یکی از بهترین صندلیا میتونم بگم هست,alia realli one best chair say,7
94456,892086,این گارد عالی و مفته...تو این بازار ک قاب واس ...,guard excel free market case phone extrem rare...,7
94606,891635,من این چرخ خیاطی رو برای مصرف خونگی خریداری ک...,bought sew machin bloodi consumpt veri satisfi,7
94918,900065,من از بازار خریدمش متاسفانه ی مقدار قیمتش روبه...,bought market unfortun price expens dj price v...,7
95949,892013,روی ابروهای من که هیچ تاثیری نذاشت.الان هم که ...,eyebrow no effect even price veri high opinion...,7
96112,899851,خیلی عالی است ولی بیرون ارزون تره,great cheaper outsid,7
97202,893102,من اسپری بدن ویکتوریا سیکرت رو قبلا از جای دیگ...,alreadi prepar victoria secret bodi spray anot...,7
97514,901875,امیدوارم قسمت همه شه,hope will everyon part,7
98796,902649,دستگاه پخش mp3 بسیار خوبی است هم USB و هم میکر...,mp3 player great support usb microsd,7


In [27]:
dat=keifiat[(keifiat['cluster labels']==8)]
dat

,product_id,comment,commenten,cluster labels
99931,907669,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8
99934,907654,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8
99935,907647,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8
99936,907634,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8


In [28]:
dat=keifiat[(keifiat['cluster labels']==9)]
dat

,product_id,comment,commenten,cluster labels
94159,890656,سلام به کساای که تو گیم pro هستند و میخان بازی...,hello game pro play pub best game amateur peop...,9
94160,890656,هرسه بخش بازی فوق العادست هم زامبی هم بتل رویا...,three part game fantast zombi battl royal mult...,9
97478,899474,بازی استراتژیک بسیار عالی که ساعت ها آدمو مشغو...,excel strategi game keep peopl busi hour excel...,9
97778,899861,خوبه من که هم بازی شو دارم هم خودشو,good game,9
98730,890656,این بازی بی نظیرترین سری مجموعه black ops است ...,game uniqu seri black op seri lot volum,9


# here is what each cluster means:
# 2good
# 5good
# 6good
# 8good
# 1Very good
# 3Very good
# 9Very good
# 7Very good
# 0excellent
# 4excellent

# and here are their meanings:
# cluster2,5,6,8->rank:1
# cluster1,3,9,7->rank:2
# cluster0,4->rank:3

In [35]:
cd=[]
for label in keifiat['cluster labels']:
    if(label==8): ll=1
    if(label==0): ll=3
    if(label==3): ll=2
    if(label==5): ll=1
    if(label==1): ll=2
    if(label==2): ll=1
    if(label==6): ll=1
    if(label==7): ll=2
    if(label==9): ll=2
    if(label==4): ll=3
    cd.append(ll)

In [36]:
keifiat['rank']=cd
keifiat

,product_id,comment,commenten,cluster labels,rank
89445,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...,hi friend phone great term hardwar veri veri s...,1,2
89485,891034,تحویل بموقع و در دسترس بودن خدمات,time deliveri avail servic,5,1
89753,891045,گرونه وبه بازار ایران دیر میرسه,expens late iranian market,7,2
89969,891312,این کاپشن از جنس بیرونی فوق العاده مناسب در بر...,jacket made outer materi extrem suitabl rain q...,1,2
90202,890725,عالیه واقعا یکی از بهترین صندلیا میتونم بگم هست,alia realli one best chair say,7,2
...,...,...,...,...,...
99934,907654,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8,1
99935,907647,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8,1
99936,907634,کیفیتش خوبه قیمتش هم مناسبه,qualiti good price reason,8,1
99952,908760,من فیکورس مدلهای اونتوس و وود رو دارم، انصافا ...,ficor ontus wood model fair bush allencompass ...,1,2


In [37]:
keifiat.groupby(by=['rank']).first()

,product_id,comment,commenten,cluster labels
rank,,,,
1,891034,تحویل بموقع و در دسترس بودن خدمات,time deliveri avail servic,5
2,891045,سلام دوستان این گوشی فوق العاده است ازنظر سخت ...,hi friend phone great term hardwar veri veri s...,1
3,894711,دیروز به دستم رسید فوق العادست,receiv yesterday wonder,0


# done.